In [ ]:
import cv2
import os
import numpy as np

def extract_frames_from_videos(videos_folder, save_folder, desired_frame_rate=5):
    os.makedirs(save_folder, exist_ok=True)

    # Get a list of all video files in the input folder
    video_files = [f for f in os.listdir(videos_folder) if f.endswith(('.mp4'))]

    for video_file in video_files:
        video_path = os.path.join(videos_folder, video_file)
        cap = cv2.VideoCapture(video_path)
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(frame_rate / desired_frame_rate)
        
        frame_number = 0
        saved_frame_count = 0

        
        video_save_folder = os.path.join(save_folder, os.path.splitext(video_file)[0])
        os.makedirs(video_save_folder, exist_ok=True)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_number % frame_interval == 0:
                resized_frame = cv2.resize(frame, (360, 360))
                
                normalized_frame = resized_frame / 255.0
                
                save_frame = (normalized_frame * 255).astype(np.uint8)
                
                save_path = os.path.join(video_save_folder, f'frame_{saved_frame_count}.jpg')
                cv2.imwrite(save_path, save_frame)
                saved_frame_count += 1

            frame_number += 1

        cap.release()
        print(f'Extracted and saved {saved_frame_count} frames from {video_file} into {video_save_folder}')

# Example usage
videos_folder = 'vid'  # Replace with your input videos folder path
save_folder = 'frames'  # Replace with your output frames folder path
extract_frames_from_videos(videos_folder, save_folder)


In [ ]:
import sys
import os

# Add the deep_sort directory to the system path
sys.path.append(os.path.join(os.getcwd(), 'deep_sort'))

from deep_sort.tracker import Tracker
from deep_sort.nn_matching import NearestNeighborDistanceMetric
from deep_sort.detection import Detection


In [ ]:
import torch

# Load YOLOv5 model (or any other YOLO variant)
model = torch.hub.load('ultralytics/yolov5:v6.2', 'yolov5s', pretrained=True)


In [ ]:
# Path to your frames (main directory containing subdirectories of frames)
frames_folder = 'frames/Discrete Trial Training/'  # Update to include subdirectories
output_folder = 'detection_outputs'

In [ ]:
import cv2
import torch
import glob
import os
from torchvision import transforms
import numpy as np
# Ensure your model is loaded and set to eval mode
model.eval()

# Define your transform (resize, normalization, etc.)
preprocess = transforms.Compose([
    transforms.ToPILImage(),  # Convert to PIL Image
    transforms.Resize((640, 640)),  # Resize to match model input
    transforms.ToTensor(),  # Convert to tensor
])
def detect_people(frame):
    # Convert the frame from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Preprocess the frame
    frame_tensor = preprocess(frame_rgb).unsqueeze(0).to('cuda')  # Add batch dimension and move to GPU

    with torch.no_grad():  # No need to calculate gradients
        results = model(frame_tensor)  # Run detection on the frame

    # Check the output format
    detections = results.cpu().numpy()  # Convert to numpy array

    return detections[0]  # Return only the first batch


# Iterate through each subdirectory in the frames folder
for subdir in glob.glob(frames_folder):
    for frame_path in glob.glob(os.path.join(subdir, '*.jpg')):  # Get all JPG files
        frame = cv2.imread(frame_path)  # Read the frame
        detections = detect_people(frame)  # Run detection on the frame

        # Draw bounding boxes on the frame
        for detection in detections:
    # Unpack the detection results
            x1, y1, x2, y2, conf = detection[:5]  # First five elements
            class_probs = detection[5:]  # Remaining elements are class probabilities
    
    # Get the index of the class with the highest probability
            cls = np.argmax(class_probs)  # Get the class index with the highest probability

    # Convert to int and format the label
            label = f'ID: {int(cls)} | Conf: {conf:.2f}'

    # Draw a rectangle around the detected object
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Use thickness of 2 for outline
            cv2.putText(frame, label, (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Save the frame with detections
        output_frame_path = os.path.join(output_folder, os.path.basename(frame_path))
        cv2.imwrite(output_frame_path, frame)

print("Detection completed and saved frames with bounding boxes.")


In [25]:
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize the YOLO model and DeepSORT
yolov8_model = YOLO('yolov8n.pt')  # Path to your YOLO model
deepsort = DeepSort(max_age=300, nn_budget=1000)



MobileNetV2 Embedder for Deep Sort initialised
- gpu enabled: True
- half precision: True
- max batch size: 16
- expects BGR: True
DeepSort Tracker initialised
- max age: 300
- appearance threshold: 0.2
- nms threshold: OFF
- max num of appearance features: 1000
- overriding track class : No
- today given : No
- in-build embedder : Yes
- polygon detections : No


In [44]:
import requests

url = "https://drive.google.com/uc?id=1dUUZ4rHDWohmsQXCRe2C_HbYkzz94iBV"
response = requests.get(url, stream=True)

with open("resnet50_market1501.pth", "wb") as f:
    f.write(response.content)

In [53]:
import torch

try:
    model = torch.load('resnet50_weights.pth')
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")


Model loaded successfully.


In [72]:
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO  # Importing YOLO from the ultralytics library
from strongsort import StrongSORT  # Importing StrongSORT tracker
import torch

# Initialize the YOLO model with YOLOv8 weights
model_weights = Path('yolov8/yolov8n.pt')  # Update this to your YOLOv8 model path
model = YOLO(model_weights)  # Load YOLOv8 model

# Set device for model
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use 'cuda' for GPU if available
fp16 = True  # Use half-precision for better performance

# Initialize StrongSORT tracker with OSNet as the ReID model
tracker = StrongSORT(model_weights='osnet_x1_0', device=device, fp16=fp16)  # OSNet is specified here

def process_videos(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for video_folder in os.listdir(input_folder):
        video_folder_path = os.path.join(input_folder, video_folder)

        if os.path.isdir(video_folder_path):
            video_output_folder = os.path.join(output_folder, video_folder)
            os.makedirs(video_output_folder, exist_ok=True)

            for img_file in sorted(os.listdir(video_folder_path)):
                if img_file.endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(video_folder_path, img_file)
                    image = cv2.imread(img_path)

                    # Perform inference with YOLO
                    results = model(image)[0]

                    # Prepare detections for StrongSORT
                    detections = []
                    for box, conf, cls in zip(results.boxes.xywh, results.boxes.conf, results.boxes.cls):
                        if cls == 0:  # Assuming you want to track only persons
                            x_center, y_center, width, height = box
                            detections.append([x_center, y_center, width, height, conf])

                    # Update the StrongSORT tracker
                    outputs = tracker.update(detections, image)

                    # Draw the tracking results on the image
                    for output in outputs:
                        track_id, bbox = output.track_id, output.tlwh
                        x1, y1, w, h = map(int, bbox)
                        cv2.rectangle(image, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
                        cv2.putText(image, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                    # Save the processed image
                    output_path = os.path.join(video_output_folder, img_file)
                    cv2.imwrite(output_path, image)
                    print(f'Processed and saved: {output_path}')

# Example usage
input_folder = 'frames/'  # Folder containing input images
output_folder = 'tracked5/'  # Folder to save output images
process_videos(input_folder, output_folder)


AttributeError: 'str' object has no attribute 'name'